# Milestone Project

The goal is to simulate the quantum adiabatic algorithm solving a small instance of maximum independence set on the graph defined by the adjacency matrix $M$, where

$$ 
M = \begin{pmatrix}
0 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 
\end{pmatrix}  
$$

### Method
1. Construct $H_{\mathrm{Ising}}$ for this graph
2. Verify that the lowest energy state is indeed the maximum independent set $|10011\rangle$, which is represented by a column vector of length $2^5$ with a one in the 19<sup>th</sup> position, and zeros elsewhere
3. Simulate an adiabatic algorithm solving the problem with a time dependent Hamiltonian 


#### Import necessary libraries

In [34]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

#### Helper functions

In [2]:
pauli_matrices = {
    'x':np.matrix(([0, 1], [1, 0]), dtype=complex),
    'y':np.matrix(([0, -1j], [1j, 0]), dtype=complex),
    'z':np.matrix(([1, 0], [0, -1]), dtype=complex)
}

def generate_sigma_j(j: int, axis: str, n:int) -> np.matrix:
    """
    Inputs:
    j: the qubit being operated on
    axis: x, y or z 
    n: number of qubits in the system
    Returns:
    Matrix representation of the axis pauli operation
    on the jth qubit
    """
    # Set pauli matrix according to input axis
    sigma = pauli_matrices[axis]
    
    # Initialise sigma_j as 1
    sigma_j = 1

    # Iterate through to n
    for i in range(n):
        if i == j:
            sigma_j = np.kron(sigma_j, sigma)
        else:
            sigma_j = np.kron(sigma_j, np.identity(2))
    
    return sigma_j

Test helper functions

In [3]:
# TODO Write helper function tests

#### 1. Construct $H_{\mathrm{Ising}}$
$$
H_{\mathrm{Ising}} = \sum_{k = 1}^n \sum_{j = k + 1} ^ n J_{kj} \sigma_k^z \sigma_j^z + \sum_{j=1}^n h_j \sigma_j^z,
$$
where $J = M$, and
$$
h_k = -\sum_{j=1}^n(M_{kj} + M_{jk}) + \kappa
$$

In [4]:
def generate_strengths(M: np.matrix, kappa:float) -> tuple[np.matrix, np.ndarray]:
    """
    Inputs are M the adjacancy matrix and kappa the variable that rewards
    more independence
    Returns J and h, the coupling and field strengths respectively
    """
    # Set n to be the number of qubits in the system
    n = M.shape[0]

    # Initialise h as an array of kappas
    h = np.ones((n), dtype=complex) * kappa

    # Generate h - TODO method could probably be vectorised
    for k in range(n):
        for j in range(n):
            h[k] += -(M[k, j] + M[j, k])
    
    return M, h

def construct_H_ising(J: np.ndarray, h: np.ndarray) -> np.matrix:
    """
    Inputs: J and h are the coupling and field strengths respectively
    Returns: The Ising Hamiltonian, a 2^n x 2^n matrix
    """
    n = h.size
    H_ising = np.zeros((2**n, 2**n), dtype=complex)

    # Add the first sum
    for k in range(n):
        for j in range(k + 1, n):
            sigma_j = generate_sigma_j(j, 'z', n)
            sigma_k = generate_sigma_j(k, 'z', n)
            H_ising += J[k, j] * sigma_k * sigma_j
    
    # Add the second sum
    for j in range(n):
        sigma_j = generate_sigma_j(j, 'z', n)
        H_ising += h[j] * sigma_j

    return H_ising

##### Test these functions with the example from the project book.

For the three qubit problem defined by the graph represented by the adjacency matrix

$$
M = \begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0 
\end{pmatrix}
$$

we can check if our functions generate the expected Ising Hamiltonian.

In [23]:
kappa_test = 0.5
M_test = np.matrix([
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0]
])
expected_h_test = np.diag(np.array([
    -2 + 3*kappa_test, 
    -2 + kappa_test,
    -2 + kappa_test, 
    2 - kappa_test,
    -2 + kappa_test,
    -2 - kappa_test,
    2 - kappa_test,
    6 - 3*kappa_test
], dtype=complex))

J_test, h_test = generate_strengths(M_test, kappa_test)

if np.array_equal(h_test, np.array([-0.5, -1.5, -0.5])):
    print("Test 1 passed! Generated correct field strengths.")
else:
    print(f"Test 1 failed, expected\n {np.array([-0.5, -1.5, -0.5])},\n got {h_test}")

my_h_test = construct_H_ising(J_test, h_test)

if np.array_equal(my_h_test, expected_h_test):
    print("Test 2 passed! Generated correct Ising Hamiltonian.")
else:
    print(f"Test failed, expected \n {expected_h_test},\n got \n {my_h_test}.")

Test 1 passed! Generated correct field strengths.
Test 2 passed! Generated correct Ising Hamiltonian.


Assuming they passed, we can generate the project Hamiltonian

In [26]:
kappa = 0.5
M = np.matrix([
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])

J, h = generate_strengths(M, kappa)

H_ising = construct_H_ising(J, h)

print(H_ising)

[[-2.5+0.j  0. +0.j  0. +0.j ...  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j -3.5+0.j  0. +0.j ...  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j -3.5+0.j ...  0. +0.j  0. +0.j  0. +0.j]
 ...
 [ 0. +0.j  0. +0.j  0. +0.j ...  9.5+0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j ...  0. +0.j  5.5+0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j ...  0. +0.j  0. +0.j 12.5+0.j]]


#### 2. Verify correct lowest energy state is in the 19th column


In [24]:
if np.min(H_ising) == H_ising[19][19]:
    print("Lowest energy state is as expected.")
else:
    print("Lowest energy state is not where we expect to find it.")

Lowest energy state is as expected.


#### 3. Simulate an adiabatic algorithm, solving the problem with a time dependent Hamiltonian

First, define the function for the time-dependent Hamiltonian,

$$
H(t) = -A(t) \sum_j \sigma_j^x + B(t)H_{\mathrm{Ising}},
$$

where $A(t) = 1 - t/t_{\mathrm{max}}$ and $B(t) = t/t_{\mathrm{max}}$ are the time-dependent controls.

In [40]:
def time_dependent_H(A: float, B: float, H_ising: np.matrix, t:float) -> np.matrix:
    """
    """
    n = H_ising.size
    H = B * H_ising

    for j in range(n):
        H -= A * generate_sigma_j(j, 'x', n)
    
    return H

def control_A(t: float, t_max: float) -> float:
    """
    Inputs:
    t: time
    t_max: total runtime of the algorithm
    Return:
    Linear time dependent control
    """
    return 1 - t/t_max

def control_B(t: float, t_max: float) -> float:
    """
    Inputs:
    t: time
    t_max: total runtime of the algorithm
    Return:
    Linear time dependent control
    """
    return t/t_max

And the time evolution operator

$$
U\left(\frac{k t_\mathrm{max}}{q}, 0\right) = \mathcal{T}\prod_{j=1}^k \exp \left ( -i \frac{t_\mathrm{max}}{q} H \left(\frac{j t_\mathrm{max}}{q}\right) \right)
$$

In [37]:
def time_evolution(t_max: float, q: int, H: np.matrix) -> np.ndarray:
    """
    Inputs:
    t_max: maximum time for evolution
    q: number of time divisions
    H: time dependent Hamiltonian evaluated at jt_max/q
    Returns:
    The evolution operator from t = j t_max / q to (j+1) t_max/q
    """
    return sc.linalg.expm(-1j * t_max / q * H)

Now, vary $t$ from $0$ to $t_{\mathrm{max}}$ in $q$ steps. 

In [41]:
qs = [50]
t_max = 100

for q in qs:
    for j in range(q):
        t = j * t_max / q

        A = control_A(t, t_max)
        B = control_B(t, t_max)

        H = time_dependent_H(A, B, H_ising, t)

        U *= time_evolution(t_max, q, H)
        


MemoryError: Unable to allocate 16.0 GiB for an array with shape (16384, 2, 16384, 2) and data type complex128